In [187]:
# %%writefile box.py
from copy import deepcopy

class Box:
    def __init__(self, x, y):
        self.coordinates = [(x, y), (x + 1, y), (x, y + 1), (x + 1, y + 1)]
        
        self.XY = (x, y)

        # lines
        self.TopLine = (self.coordinates[0], self.coordinates[1])
        self.LeftLine = (self.coordinates[0], self.coordinates[2])
        self.RightLine = (self.coordinates[1], self.coordinates[3])
        self.BottomLine = (self.coordinates[2], self.coordinates[3])
        # lines 
        self.lines = [self.TopLine, self.LeftLine, self.RightLine, self.BottomLine]

        # lines connection indicator 
        self._top = False
        self._left = False
        self._right = False
        self._bottom = False

        self.owner = None
        self.completed = False

        self.value = 1

    def connect(self, coordinates):
        line = coordinates
        success = False

        if line not in self.lines:
            return False
        
        if line == self.TopLine and self._top is False:
            self._top = True
            success = True
        elif line == self.LeftLine and self._left is False:
            self._left = True
            success = True
        elif line == self.RightLine and self._right is False:
            self._right = True
            success = True
        elif line == self.BottomLine and self._bottom is False:
            self._bottom = True
            success = True

        if self._top == True and self._bottom == True and self._left == True and self._right == True:
            self.completed = True
        
        return success

    def copy(self):
        return deepcopy(self)
    
    def _repr_pretty_(self, p, cycle):
        if cycle:
            pass

        if self._top:
            p.text("*---*")
        else:
            p.text("*   *")
        p.break_()
        if self._left:
            p.text("|")
        else:
            p.text(" ")
        
        if self.completed:
            p.text(f" {self.owner} ")
        else:
            p.text("   ") 

        if self._right:
            p.text("|")

        p.break_()

        if self._bottom:
            p.text("*---*")
        else:
            p.text("*   *")



In [180]:
# %%writefile board.py
from collections import deque

class Board:
    def __init__(self, m, n):
        self.playerScore = 0
        self.aiScore = 0
        self.m = m
        self.n = n
        self._boxes = self.generateBoxes(m, n)
        self._openVectors = self.generateVectors(m, n)
        self._connectedVectors = set()

    def generateBoxes(self, rows, cols):
        """
        This function generates the boxes of the board
        """
        boxes = [[Box(x, y) for x in range(cols)] for y in range(rows)]

        return boxes
    
    def generateVectors(self, m, n):
        '''
        The vectors represent all of the available moves, or lines, which can
        be played on a game board of m rows and n columns. These are stored as tuples
        containing each coordinate and are stored in a queue. The vector queue, along
        with the list of boxes that correspond to the coordinates, are used to represent
        game state.
        Vector format: ((x1, y1), (x2, y2))
        '''
        vectors = deque()
        for i in range(0, m + 1):
            for j in range(0, n):
                # Adding horizontal line vectors
                vectors.append(((j, i), (j + 1, i)))
                # Adding vertical line vectors if not in the last row
                if i < m:
                    vectors.append(((j, i), (j, i + 1)))
            # Adding the vertical line for the last column in the current row
            if i < m:
                vectors.append(((n, i), (n, i + 1)))
        return vectors

    def move(self, coordinates, player_move: bool = False):
        player = "P" if player_move == True else "A"
        
        if coordinates in self._openVectors:
            self._openVectors.remove(coordinates)
            self._connectedVectors.add(coordinates)
            self.checkboxes(coordinates, player)
        
    def checkboxes(self, coordinates, player: str):
        for i in range(self.m):
            for j in range(self.n):
                box = self._boxes[i][j]
                if coordinates in box.lines:
                    box.connect(coordinates)
                if box.completed == True and box.owner == None:
                    box.owner = player
                    self.prevComplete = True
                    if player == "P":
                        self.playerScore += 1
                    else:
                        self.aiScore += 1
    
    def displayBoard(self):
        '''
        Displays the current board state in the terminal, including the grid of dots,
        lines (representing player moves), and box ownership. Also displays the scores
        of Player 1 and the AI.
        '''
        # Display player scores
        print(f"Player 1: {self.playerScore}")
        print(f"Player AI: {self.aiScore}\n")

        # Display the board
        for i in range(self.m):
            # Top line of each row
            top_line = "   "  # Start with some spacing for alignment
            middle_line = "   "  # Line below to display vertical lines and boxes
            
            for j in range(self.n):
                # Dot
                top_line += "*"

                # Horizontal line
                if ((j, i), (j + 1, i)) in self._connectedVectors:
                    top_line += "---"
                else:
                    top_line += "   "
                
                # Vertical line and box
                if ((j, i), (j, i + 1)) in self._connectedVectors:
                    middle_line += "|"
                else:
                    middle_line += " "
                
                # Box ownership
                if self._boxes[j][i].completed:
                    middle_line += f" {self._boxes[j][i].owner} "
                else:
                    middle_line += "   "
            
            # Last dot on the right end of the row
            top_line += "*"
            if ((self.n, i), (self.n, i + 1)) in self._connectedVectors:
                middle_line += "|"
            else:
                middle_line += " "
            
            # Print the top line and the middle line
            print(top_line)
            print(middle_line)

        # Bottom line of the last row
        bottom_line = "   "
        for j in range(self.n):
            bottom_line += "*"
            if ((j, self.m), (j + 1, self.m)) in self._connectedVectors:
                bottom_line += "---"
            else:
                bottom_line += "   "
        bottom_line += "*"
        
        # Print the bottom line
        print(bottom_line)
        print("")  # New line for spacing
    
    def _repr_pretty_(self, p, cycle):
        if (cycle):
            pass

        display_single_box = True

        # Display player scores
        p.text(f"    AI Score : {self.aiScore}\n")
        p.text(f"Player Score : {self.playerScore}\n")

        if display_single_box:
            self.__single_box(p)
            return
        
        for i in range(self.m):
            top_line = "\t"
            middle_line = "\t"
            bottom_line = "\t"

            for j in range(self.n):
                box = self._boxes[i][j]
                top_line += "*---*" if box._top else "*   *"
                middle_line += "|" if box._left else " "
                middle_line += " X " if box.completed else "   "
                middle_line += "|" if box._right else " "
                bottom_line += "*---*" if box._bottom else "*   *"
            
            p.text(top_line)
            p.break_()
            p.text(middle_line)
            p.break_()
            p.text(bottom_line)
            p.break_()

    def __single_box(self, p):
        last_line = ""
        for i in range(self.m - 1):
            top_line = "\t*"
            middle_line = "\t"
            last_line = "\t*"

            for j in range(self.n):
                box = self._boxes[i][j]
                top_line += "---*" if box._top else "   *"
                middle_line += "|" if box._left else " "
                if box.completed:
                    middle_line += f" {box.owner} "
                else:
                    middle_line += "   "
                last_line += "---*" if box._top else "   *"

            
            if box and box._right:
                middle_line += "|"
            
            p.text(top_line)
            p.break_()
            p.text(middle_line)
            p.break_()

        p.text(last_line)
        p.break_()
            

        # # Display the board
        # for i in range(self.m):
        #     # Top line of each row
        #     top_line = "   "  # Start with some spacing for alignment
        #     middle_line = "   "  # Line below to display vertical lines and boxes
            
        #     for j in range(self.n):
        #         # Dot
        #         top_line += "*"

        #         # Horizontal line
        #         if ((j, i), (j + 1, i)) in self._connectedVectors:
        #             top_line += "---"
        #         else:
        #             top_line += "   "
                
        #         # Vertical line and box
        #         if ((j, i), (j, i + 1)) in self._connectedVectors:
        #             middle_line += "|"
        #         else:
        #             middle_line += " "
                
        #         # Box ownership
        #         if self._boxes[j][i].completed:
        #             middle_line += f" {self._boxes[j][i].owner} "
        #         else:
        #             middle_line += "   "
            
        #     # Last dot on the right end of the row
        #     top_line += "*"
        #     if ((self.n, i), (self.n, i + 1)) in self._connectedVectors:
        #         middle_line += "|"
        #     else:
        #         middle_line += " "
            
        #     # Print the top line and the middle line
        #     p.text(top_line)
        #     p.break_()
        #     p.text(middle_line)
        #     p.break_()

        # # Bottom line of the last row
        # bottom_line = "   "
        # for j in range(self.n):
        #     bottom_line += "*"
        #     if ((j, self.m), (j + 1, self.m)) in self._connectedVectors:
        #         bottom_line += "---"
        #     else:
        #         bottom_line += "   "
        # bottom_line += "*"
        
        # # Print the bottom line
        # p.text(bottom_line)
    

        
    
    # def displayBoard__1(self):
    #     '''
    #     This function generates a text-based representation of the current board state
    #     to be displayed on the command line. The display is based on the row & column
    #     values stored on the board, along with the connectedVectors set objects, which
    #     stores previously connected dots.
    #     '''
    #     print("Player 1: %s" % self.playerScore)
    #     print("Player AI: %s" % self.aiScore)
    #     # Set X axis Labels
    #     str1 = "\n  "
    #     for i in range(self.m + 1):
    #         str1 = str1 + "   %s" % i
    #     print(str1)

    #     # Draw remaining board
    #     boxVal = " "
    #     for i in range(self.m + 1):
    #         # Append the Y axis label to the beginning of a row
    #         str1 = "%s " % i
    #         str2 = "     "
    #         for j in range(self.n + 1):
    #             # Check for horizontal connections
    #             if ((j - 1, i), (j, i)) in self._connectedVectors:
    #                 str1 = str1 + "---*"
    #             else:
    #                 str1 = str1 + "   *"

    #             # Check for the box value of a given square based on the top left coordinate
    #             if j < self.n:
    #                 if self._boxes[j][i - 1].XY == (j, i - 1):
    #                     boxVal = self._boxes[j][i - 1].value
    #             else:
    #                 boxVal = " "

    #             # Check for vertical connections
    #             if ((j, i - 1), (j, i)) in self._connectedVectors:
    #                 str2 = str2 + "| %s " % boxVal
    #             else:
    #                 str2 = str2 + "  %s " % boxVal
    #         print(str2)
    #         print(str1)
    #     print("")

    # def displayBoard__2(self):
    #     '''
    #     Generates a text-based representation of the current board state to be displayed
    #     on the command line. The display shows the board grid, the connections between dots,
    #     and the current scores of the players.
    #     '''
    #     # Display player scores
    #     print(f"Player 1: {self.playerScore}")
    #     print(f"Player AI: {self.aiScore}\n")

    #     # Display X-axis labels
    #     x_labels = "  " + "".join(f"   {i}" for i in range(self.m + 1))
    #     print(x_labels)

    #     # Draw the board
    #     for i in range(self.m + 1):
    #         row_str = f"{i} "  # Y-axis label
    #         line_below_str = "    "

    #         for j in range(self.n + 1):
    #             # Horizontal connection
    #             if ((j - 1, i), (j, i)) in self._connectedVectors:
    #                 row_str += "---*"
    #             else:
    #                 row_str += "   *"

    #             # Box value (between rows)
    #             box_value = " "
    #             if j < self.n and i > 0:
    #                 if self._boxes[j][i - 1].XY == (j, i - 1):
    #                     box_value = self._boxes[j][i - 1].value

    #             # Vertical connection
    #             if ((j, i - 1), (j, i)) in self._connectedVectors:
    #                 line_below_str += f"| {box_value} "
    #             else:
    #                 line_below_str += f"  {box_value} "

    #         # Print the row and the line below it
    #         print(line_below_str)
    #         print(row_str)

    #     print("")


In [161]:
Board(3, 3)

    AI Score : 0
Player Score : 0
	*   *   *   *
	            
	*   *   *   *
	            
	*   *   *   *


In [181]:
# from board import Board

board = Board(5, 5)
board

    AI Score : 0
Player Score : 0
	*   *   *   *   *   *
	                    
	*   *   *   *   *   *
	                    
	*   *   *   *   *   *
	                    
	*   *   *   *   *   *
	                    
	*   *   *   *   *   *


In [185]:
board.move(((0, 0), (0, 1)), player_move=True)
board

    AI Score : 0
Player Score : 1
	*---*   *   *   *   *
	| P |               
	*---*   *   *   *   *
	                    
	*   *   *   *   *   *
	                    
	*   *   *   *   *   *
	                    
	*   *   *   *   *   *


In [188]:
board._boxes[0][0]

*---*
| X |
*---*